<a href="https://colab.research.google.com/github/Richards-R/FCC-Neural-Network-SMS-Text-Classifier/blob/main/Richards_R_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

!pip install tensorflow-datasets
import tensorflow_datasets as tfds

print(tf.__version__)

2.15.0


In [2]:
vocab_size = 30000
embedding_dim = 16
max_length = 150
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-01-03 13:10:13--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2024-01-03 13:10:13 (10.5 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-01-03 13:10:13--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [4]:
train_df = pd.read_csv(train_file_path, sep="\t", header=None)
test_df = pd.read_csv(test_file_path, sep="\t", header=None)


In [5]:
train_df.shape

(4179, 2)

In [6]:
#Preprocess & clean
train_df = train_df.drop_duplicates()

In [7]:
train_df.dropna(inplace = True)
test_df.dropna(inplace = True)

<ipython-input-7-cf7481d8e199>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.dropna(inplace = True)


In [8]:
train_df.shape

(3935, 2)

In [9]:
train_labels = train_df[0]
test_labels = test_df[0]

In [10]:
train_labels[train_labels == 'ham'] = 0
train_labels[train_labels == 'spam'] = 1
test_labels[test_labels == 'ham'] = 0
test_labels[test_labels == 'spam'] = 1

<ipython-input-10-b1a1a3e88bbd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_labels[train_labels == 'ham'] = 0
<ipython-input-10-b1a1a3e88bbd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_labels[train_labels == 'spam'] = 1


In [11]:
train_labels = train_labels.tolist()
test_labels = test_labels.tolist()

In [12]:
train_messages = train_df[1].tolist()
test_messages = test_df[1].tolist()

In [13]:
print(train_messages)
print(test_messages)

['ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.', 'you can never do nothing', 'now u sound like manky scouse boy steve,like! i is travelling on da bus home.wot has u inmind 4 recreation dis eve?', 'mum say we wan to go then go... then she can shun bian watch da glass exhibition...', 'never y lei... i v lazy... got wat? dat day ü send me da url cant work one...', 'in xam hall boy asked girl tell me the starting term for dis answer i can den manage on my own after lot of hesitation n lookin around silently she said the! intha ponnungale ipaditan;)', "genius what's up. how your brother. pls send his number to my skype.", 'they finally came to fix the ceiling.', 'urgent! call 09066350750 from your landline. your complimentary 4* ibiza holiday or 10,000 cash await collection sae t&cs po box 434 sk3 8wp 150 ppm 18+', 'now that you have started dont stop. just pray for more good ideas and anyt

In [14]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from functools import reduce
porter_stemmer = PorterStemmer()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [15]:
# Tokenize and stem training message input
train_df['tokenized_sentence'] = train_df.apply(lambda row: nltk.word_tokenize(str(row[1])), axis=1)
train_df[1] = train_df['tokenized_sentence'].apply(lambda x : [porter_stemmer.stem(y) for y in x])

In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding

In [17]:
tokenizer = Tokenizer(oov_token = oov_tok, filters='!"#$%&()*+,-./:;<=>@[\]^_`{|}~ ')
tokenizer.fit_on_texts(train_messages)
word_index = tokenizer.word_index

In [18]:
train_sequences = tokenizer.texts_to_sequences(train_messages)
train_padded = pad_sequences(train_sequences)

test_sequences = tokenizer.texts_to_sequences(test_messages)
test_padded = pad_sequences(test_sequences)

In [19]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [20]:
epochs = 3
# Create model
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, 64),
  tf.keras.layers.LSTM(128),
  tf.keras.layers.Dense(256, activation="relu"),
  tf.keras.layers.Dense(128, activation="relu"),
  tf.keras.layers.Dense(1, activation="sigmoid")
])



In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          1920000   
                                                                 
 lstm (LSTM)                 (None, 128)               98816     
                                                                 
 dense (Dense)               (None, 256)               33024     
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2084865 (7.95 MB)
Trainable params: 2084865 (7.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [23]:
history = model.fit(train_padded, train_labels, epochs=epochs, batch_size=128, validation_data=(test_padded, test_labels))

Epoch 1/3
31/31 [==============================] - 34s 790ms/step - loss: 0.3456 - accuracy: 0.8648 - val_loss: 0.1691 - val_accuracy: 0.9677
Epoch 2/3
31/31 [==============================] - 23s 720ms/step - loss: 0.0792 - accuracy: 0.9860 - val_loss: 0.0499 - val_accuracy: 0.9864
Epoch 3/3
31/31 [==============================] - 23s 758ms/step - loss: 0.0140 - accuracy: 0.9967 - val_loss: 0.0423 - val_accuracy: 0.9849


In [24]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  # Tokenize words in message
  words = word_tokenize(pred_text)

  # Using reduce to apply stemmer to each word and join them back into a string
  stemmed_text = reduce(lambda x, y: x + " " + porter_stemmer.stem(y), words, "")

  # Append each word to a list
  stemmed_text_list = []
  stemmed_text_list.append(stemmed_text.split())

  # Tokenize the list into sequence
  sequences = tokenizer.texts_to_sequences(stemmed_text_list)

  # Pad each sequence to make each length uniform
  padded = pad_sequences(sequences)

  # Calculate probability the padded sequence is spam
  probability = model.predict(padded)[0][0]

  # Format return message
  prediction = [probability, "ham" if probability < 0.5 else "spam"]

  return (prediction)

pred_text = "how are you doing today"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 1s 535ms/step
[0.01665586, 'ham']


In [25]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    print("Message: ", msg, "Class: ", ans)
    prediction = predict_message(msg)
    print(prediction)

    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


Message:  how are you doing today Class:  ham
1/1 [==============================] - 0s 27ms/step
[0.01665586, 'ham']
Message:  sale today! to stop texts call 98912460324 Class:  spam
1/1 [==============================] - 1s 525ms/step
[0.98753816, 'spam']
Message:  i dont want to go. can we try it a different day? available sat Class:  ham
1/1 [==============================] - 0s 28ms/step
[0.0005526922, 'ham']
Message:  our new mobile video service is live. just install on your phone to start watching. Class:  spam
1/1 [==============================] - 0s 31ms/step
[0.9861538, 'spam']
Message:  you have won £1000 cash! call to claim your prize. Class:  spam
1/1 [==============================] - 0s 25ms/step
[0.99996907, 'spam']
Message:  i'll bring it tomorrow. don't forget the milk. Class:  ham
1/1 [==============================] - 0s 27ms/step
[0.0028948658, 'ham']
Message:  wow, is your arm alright. that happened to me one time too Class:  ham
1/1 [===========================